In [15]:
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
movies_df = pd.read_csv('./data/movies_data.csv')
movies_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,PlotSummary,movie_text
0,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...,"Boone's cabin is attacked by the Indians, who ...",Title: Daniel Boone\nRelease Year: 1907\nDirec...
1,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...,A father and mother take their daughter Dollie...,Title: The Adventures of Dollie\nRelease Year:...
2,1908,The Black Viper,American,D. W. Griffith,D. W. Griffith,drama,https://en.wikipedia.org/wiki/The_Black_Viper,A thug accosts a girl as she leaves her workpl...,A thug accosts a girl as she leaves her workpl...,Title: The Black Viper\nRelease Year: 1908\nDi...
3,1908,The Call of the Wild,American,D. W. Griffith,Charles Inslee,adventure,https://en.wikipedia.org/wiki/The_Call_of_the_...,A white girl (Florence Lawrence) rejects a pro...,Florence Lawrence was already the most popular...,Title: The Call of the Wild\nRelease Year: 190...
4,1908,A Christmas Carol,American,Unknown,Tom Ricketts,drama,https://en.wikipedia.org/wiki/A_Christmas_Caro...,No prints of the first American film adaptatio...,No prints of the first American film adaptatio...,Title: A Christmas Carol\nRelease Year: 1908\n...


In [9]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

In [13]:
chunks = []
chunk_source = []  # Для отслеживания источника чанка


for idx, row in movies_df.iterrows():
    movie_chunks = splitter.split_text(row['movie_text'])
    chunks.extend(movie_chunks)
    chunk_source.extend([idx] * len(movie_chunks))

print(f"Total chunks: {len(chunks)}")

Total chunks: 61107


In [23]:
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [35]:
embeddings = model.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/1910 [00:00<?, ?it/s]

In [36]:
embeddings[0].shape

(768,)

In [37]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [46]:
faiss.write_index(index, "./data/movies_info.index")


In [49]:
# index = faiss.read_index("movies_info.index")
# index

In [41]:
chunk_metadata = {
    'chunks': chunks,
    'source_index': chunk_source,
    'titles': movies_df['Title'].iloc[chunk_source].values,
    'movie_text': movies_df['movie_text'].iloc[chunk_source].values
}
import pickle
with open('./data/chunks_metadata.pkl', 'wb') as f:
    pickle.dump(chunk_metadata, f)

In [42]:
query_embedding = model.encode(['space adventure'])

distances, indices = index.search(query_embedding, 5)

In [43]:
distances

array([[28.258404, 29.332535, 29.83104 , 30.324926, 31.512108]],
      dtype=float32)

In [44]:
indices

array([[37389, 35590, 22737, 60258, 59478]])